## Tensor Polyhedra Framework  
### Welcome to the future.  
This is an educational version using symbolic library to focus on linear transformation. For a production code symbolic calculation is not needed.  
In this demo we are going to perform a Polyhedral Scheduler Transformation on the following kernel written in C:  
```
for(i=0;i<=N;i++)
    A[i]=0             // X statement
for(j=0;j<=N;j++)
    for(k=0,k<=N;k++)
        B[j] += A[k];  // Y Statement
```  
In a polyhedral transformation, we do not care of the computing statements, so the above Kernel can be rewritten as:  
```
for(i=0;i<=N;i++)
    X(i);             
for(j=0;j<=N;j++)
    for(k=0,k<=N;k++)
        Y(j,k);
```  
Let's introduce the concept of *scheduling vectors*. A *scheduling vector* captures information regarding *when* a tuple $(i,j,k)$ is executed. Hence whe can map our ```X``` and ```Y``` statements like this:  
```X(i,0,0)```-> X[i]     is executed at _date_ [i * *days*,0,0]  
```Y(N,j,k)```-> Y[N,j,k] is executed at _data_ [N * *days*, j * *minutes*, k * *seconds*]  

So, what really *is* a Polyhedral Transformation?  
Well, unfortunatly Polyhedral Theory is difficult to understand, but the underlying idea is quit straight forward. How can we rearrange X and Y execution dates to have a new Kernel which obtain the same result as source Kernel but takes less time to obtain it?  
This is done finding new X' and Y' such as  

$$X´=(t^0_{xi}.i+t^0_{xj}.j+t^0_{xk}.k+t^0_{xN}.N+t^0_{x1}.1,t^1_{xi}.i+t^1_{xj}.j+t^1_{xk}.k+t^1_{xN}.N+t^1_{x1}.1)$$ and  

$$Y´=(t^0_{yi}.i+t^0_{yj}.j+t^0_{yk}.k+t^0_{yN}.N+t^0_{y1}.1,t^0_{yi}.i+t^0_{yj}.j+t^0_{yk}.k+t^0_{yN}.N+t^0_{y1}.1)$$

where $t^d_{sr}$ coefficients are optimaly calculated to ensure that we get a code that yields the same result and execute faster relying on the possibilities of parallelization of the hardware.  
In order to solve our problem, we need to feed our polyhedral model with a set of constraints that ensures legal excution of code:  
we define:  
```
    D_X = { (i)     | 0 <= i <= N }  
    D_Y = { (j,k)   | 0 <= j <= N && 0 <= k <= N }
```
We can then define a kind of constraints called *dependencies* such as:

```  
    D_XY = { (ic,jc,kc)     | 0 <= ic <= N  && 0 <= jc <= N && 0 <= kc <= N && ic == kc } 
    D_YY = { (js,ks,jt,kt)  | 0 <= js <= N  && 0 <= ks <= N && 0 <= jt <= N  && 0 <= kt <= N && js == kt && kt > ks  }
```
Expression ```D_XY``` captures the read/write access of statements X and Y over array A: ```A[k]=A[i]```.  
In the other hand, expression ```D_YY``` capures the read/write access of statement Y over array B: ```B[j] = B[j] + A[k]```.

Last, let's define two function $f$ and $g$ such as:  

```
    f(i_c,j_c,k_t)=T_y(j_c,k_c)-T_x(i_c)-e_dxy>=0 in D_XY
```
and  
```    
    g(j_s,k_s,j_t,k_t)=T_y(j_t,k_t)-T_y(j_s,k_s)-e_dyy>=0 in D_YY
``` 

Both functions captures the access relations and the precedences constraints. From C code we derive that B[k] must be executed after A[i]. Besides variables js,ks,jt and kt captures the B[k] = B[k] + A[j] precedence constraint.

SyntaxError: invalid syntax (<ipython-input-1-4408bb309bdb>, line 1)